In [1]:
# show the matplotlib plotted graph within notebook lines.
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [4]:
# import required libraries
# pandas:Data framework library for Python
# sklearn: Library to perform machine learning tasks
import os
import time
from tqdm import tqdm
import pandas as pd
import numpy as np
import scipy
import scipy.stats as st
import re
import codecs
from collections import Counter
import nltk
from nltk.stem import *
from nltk.corpus import stopwords
import sklearn 
import sklearn.datasets
import sklearn.metrics as metrics 
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from scipy import stats

In [5]:
# extract data
!gzip -d -k 20news-19997.tar.gz

!tar -xf 20news-19997.tar

In [6]:
!ls 20_newsgroups

alt.atheism
comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x
misc.forsale
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey
sci.crypt
sci.electronics
sci.med
sci.space
soc.religion.christian
talk.politics.guns
talk.politics.mideast
talk.politics.misc
talk.religion.misc


In [7]:
def getFilesDirectory(datapath='20_newsgroups'):
    # create file directory for all files
    files = []
    for (path, dirnames, filenames) in os.walk(datapath):
        files.extend(os.path.join(path, name) for name in filenames)
    # putting file directories into pandas dataframe
    directorydf= pd.DataFrame(files)
    directorydf.columns = ['Directories']
    return directorydf

In [8]:
directorydf = getFilesDirectory()

In [9]:
def label_target (d):
    if d.find("alt.atheism") > 0 :
      return 0
    if d.find("comp.graphics") > 0:
      return 1
    if d.find("comp.os.ms-windows.misc") > 0:
      return 2
    if d.find("comp.sys.ibm.pc.hardware") > 0:
      return 3
    if d.find("comp.sys.mac.hardware") > 0:
      return 4
    if d.find("comp.windows.x") > 0:
      return 5
    if d.find("misc.forsale") > 0:
      return 6
    if d.find("rec.autos") > 0:
      return 7
    if d.find("rec.motorcycles") > 0:
      return 8
    if d.find("rec.sport.baseball") > 0:
      return 9
    if d.find("rec.sport.hockey") > 0:
      return 10
    if d.find("sci.crypt") > 0:
      return 11
    if d.find("sci.electronics") > 0:
      return 12
    if d.find("sci.med") > 0:
      return 13
    if d.find("sci.space") > 0:
      return 14
    if d.find("soc.religion.christian") > 0:
      return 15
    if d.find("talk.politics.guns") > 0:
      return 16
    if d.find("talk.politics.mideast") > 0:
      return 17
    if d.find("talk.politics.misc") > 0:
      return 18
    if d.find("talk.religion.misc") > 0:
      return 19
    return 'Other'

In [10]:
# putting file directories into pandas dataframw
directorydf.columns = ['Directories']
directorydf.head(n=5)

,Directories
0,20_newsgroups\alt.atheism\49960
1,20_newsgroups\alt.atheism\51060
2,20_newsgroups\alt.atheism\51119
3,20_newsgroups\alt.atheism\51120
4,20_newsgroups\alt.atheism\51121


In [11]:
def dataPreprocessor(k, fs, encode, directorydf, size=None):
    """
    INPUT
        k: int. number of features to use
        fs: string. From ['tf', 'mi']
        encode: string. From ['tf', 'boolean']
        drectorydf: Dataframe. It is given, see above support functions
        size: int. Sample size. Default should be the data size.
    OUTPUT
        data: Dataframe. preprocessed data
    
    ps: 'tf' means term frequency, 'mi' means mutual information
    """
    
    ########### your code goes here ###########
    nltk.download('stopwords')
    from collections import Counter
    import re
    import codecs
    counter = Counter()
    # Open the files and count the word frequency in each file in a loop and update the counter after finished processing a file
    for rownum, row in enumerate(directorydf.itertuples()):
        with codecs.open(row.Directories,"r" ,encoding='utf-8', errors='ignore') as myfile:
            counter.update([word.lower() for word in re.findall(r'\w+', myfile.read())if word not in stopwords.words('english')])
        if (rownum % 1000 == 0):
            print("processed %d files" % (rownum+1))
        if (rownum == size):
            break
            
    topk = counter.most_common(k)
            
    np = []
    if encode == 'boolean':
    # now we had top k words, count the frequecy (binary) of these words in individual file
        for rownum, row in enumerate(directorydf.itertuples()):
            with codecs.open(row.Directories,"r" ,encoding='utf-8', errors='ignore') as myfile:
                tempCounter = Counter([word for word in re.findall(r'\w+', myfile.read())])
                # if the word appears in the doc, then 1, else 
                topkinDoc = [1 if tempCounter[word] > 0 else 0 for (word,wordCount) in topk]
                # create a list for top k words with encoded target and its label
                np.append(topkinDoc+[label_target(row.Directories)])
                if (rownum % 1000 == 0):
                    print("processed %d files" % (rownum+1))
                if (rownum == size):
                    break
    else:
        for rownum, row in enumerate(directorydf.itertuples()):
            with codecs.open(row.Directories,"r" ,encoding='utf-8', errors='ignore') as myfile:
                tempCounter = Counter([word for word in re.findall(r'\w+', myfile.read())])
                # if the word appears in the doc, then 1, else 
                topkinDoc = [tempCounter[word] if tempCounter[word] > 0 else 0 for (word,wordCount) in topk]
                # create a list for top k words with encoded target and its label
                np.append(topkinDoc+[label_target(row.Directories)])
                if (rownum % 1000 == 0):
                    print("processed %d files" % (rownum+1))
                if (rownum == size):
                    break
            
                
    data = pd.DataFrame(np)
    dfName = []
    for c in topk:
        dfName.append(c[0])
    dfName
    data.columns = dfName+['target']
    ###########         end         ###########
    return data

In [13]:
#Might get back to this function to make it more efficient
data = dataPreprocessor(k=50, fs='tf', encode='tf', directorydf=directorydf, size=10)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
processed 1 files
processed 1 files


In [14]:
data

,edu,god,atheism,atheists,the,i,religious,many,cmu,religion,...,existence,would,say,gap,cantaloupe,state,uk,for,net,target
0,10,3,15,0,63,1,2,3,5,2,...,4,0,0,0,2,2,5,14,1,0
1,5,1,49,65,161,0,42,26,4,23,...,10,7,11,0,2,9,5,38,1,0
2,7,0,1,0,31,0,0,0,3,0,...,0,4,1,0,1,0,0,4,1,0
3,6,0,2,0,9,0,1,0,4,2,...,0,0,0,0,2,2,2,1,1,0
4,4,0,2,0,2,0,0,0,4,0,...,0,0,0,0,2,0,0,1,1,0
5,5,4,2,0,32,0,0,0,3,1,...,0,4,3,0,1,0,0,5,1,0
6,13,0,2,1,3,0,0,0,3,0,...,0,0,0,4,1,0,0,0,2,0
7,8,1,1,0,7,0,0,0,3,0,...,0,0,0,0,1,0,0,2,1,0
8,9,0,1,0,19,0,0,0,3,0,...,0,1,1,2,1,1,0,1,1,0
9,14,0,1,0,1,0,0,0,3,0,...,0,0,0,5,1,0,0,0,2,0
